In [1]:
import pandas as pd
import altair as alt

# Lendo os dados brutos

In [2]:
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')
dados_federais_campina = pd.read_csv("data\\dados_federais_vacinacao_cg.csv", encoding='UTF-8', error_bad_lines=False)

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Separando os dados de aplicação da vacina por dose da base federal.

# Primeira Dose
df_campina1Dose = dados_federais_campina.loc[dados_federais_campina['vacina_descricao_dose'] == '1ª Dose']
df_campina1Dose = df_campina1Dose.loc[df_campina1Dose['vacina_codigo'] != 88]
df_1DosePorDia = df_campina1Dose['vacina_dataaplicacao'].value_counts()

# Segunda Dose
df_campina2Dose = dados_federais_campina.loc[dados_federais_campina['vacina_descricao_dose'] == '2ª Dose']
df_campina2Dose = df_campina2Dose.loc[df_campina2Dose['vacina_codigo'] != 88]
df_2DosePorDia = df_campina2Dose['vacina_dataaplicacao'].value_counts()

# Dose Única
df_campinaDoseUnica = dados_federais_campina.loc[dados_federais_campina['vacina_codigo'] == 88]
df_DoseUnicaPorDia = df_campinaDoseUnica['vacina_dataaplicacao'].value_counts()

In [25]:
# Ordenando os dados pela data
df_1DosePorDia     = df_1DosePorDia.sort_index()
df_2DosePorDia     = df_2DosePorDia.sort_index()
df_DoseUnicaPorDia = df_DoseUnicaPorDia.sort_index()

In [26]:
# Separando os dados de aplicação da vacina por dose da base municipal.

# Primeira Dose
dm_1DosePorDia = dados_municipais_campina['vacinadosDose1Dia']
#dm_1DosePorDia = dm_1DosePorDia.loc[df_1DosePorDia.index]

# Segunda Dose
dm_2DosePorDia = dados_municipais_campina['vacinadosDose2Dia']
#dm_2DosePorDia = dm_2DosePorDia.loc[df_2DosePorDia.index]

# Dose Única
dm_DoseUnicaPorDia = dados_municipais_campina['vacinadosDoseUnicaDia']
dm_DoseUnicaPorDia = dm_DoseUnicaPorDia.loc[df_DoseUnicaPorDia.index]


In [27]:
# Primeira Dose
vacinados1DosePorDia = pd.DataFrame({
  'data': df_1DosePorDia.keys().tolist(),
  'Federal': df_1DosePorDia.values.tolist(),
  'Municipal': dm_1DosePorDia.values.tolist()
}).melt(id_vars=['data'], value_vars=['Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [28]:
alt.Chart(vacinados1DosePorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Vacinados'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=700,
    title='Quantidade de vacinados com 1º Dose por Dia'
)

alt.Chart(...)

In [29]:
# Segunda Dose
vacinados2DosePorDia = pd.DataFrame({
  'data': df_2DosePorDia.keys().tolist(),
  'Federal': df_2DosePorDia.values.tolist(),
  'Municipal': dm_2DosePorDia.values.tolist()
}).melt(id_vars=['data'], value_vars=['Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [30]:
alt.Chart(vacinados2DosePorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Vacinados'),
    alt.Color('fonte_dados:N', title='Base de Dados')
).properties(
    width=700,
    title='Quantidade de vacinados com 2º Dose por Dia'
)

alt.Chart(...)

In [31]:
# Dose Única
vacinadosDoseUnicaPorDia = pd.DataFrame({
  'data': df_DoseUnicaPorDia.keys().tolist(),
  'Federal': df_DoseUnicaPorDia.values.tolist(),
  'Municipal': dm_DoseUnicaPorDia.values.tolist()
}).melt(id_vars=['data'], value_vars=['Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [32]:
alt.Chart(vacinadosDoseUnicaPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Vacinados'),
    alt.Color('fonte_dados:N', title='Base de Dados')
).properties(
    width=600,
    title='Quantidade de vacinados com Dose Única por Dia'
)

alt.Chart(...)

In [33]:
total1Dose = df_1DosePorDia.values.sum()
total2Dose = df_2DosePorDia.values.sum()
totalDoseUnica = df_DoseUnicaPorDia.sum()

In [34]:
df_vacinadosTotalPorDose = pd.DataFrame({
    'dose': ['1Dose', '2Dose', 'Dose'],
    'aplicacoes': [total1Dose, total2Dose, totalDoseUnica]
})

In [35]:
dm_vacinadosTotalPorDose = dados_municipais_campina[['vacinadosDose1','vacinadosDose2', 'vacinadosDoseUnica']]
dm_vacinadosTotalPorDose = pd.DataFrame({
    'dose': dm_vacinadosTotalPorDose.iloc[-1].index.tolist(),
    'aplicacoes': (dm_vacinadosTotalPorDose.iloc[-1].values - dm_vacinadosTotalPorDose.iloc[0].values).tolist()
})

In [36]:
primeiraDoseTotal = pd.DataFrame({
    'base': ['Municipal', 'Federal'],
    'total': [dm_vacinadosTotalPorDose['aplicacoes'][0], df_vacinadosTotalPorDose['aplicacoes'][0]]
})

alt.Chart(primeiraDoseTotal).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Vacinados'),
    alt.Color('base:N', title='Base')
).properties(
    width=150,
    title='Total de aplicações da Primeira Dose'
)

alt.Chart(...)

In [37]:
segundaDoseTotal = pd.DataFrame({
    'base': ['Municipal', 'Federal'],
    'total': [dm_vacinadosTotalPorDose['aplicacoes'][1], df_vacinadosTotalPorDose['aplicacoes'][1]]
})

alt.Chart(segundaDoseTotal).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Vacinados'),
    alt.Color('base:N', title='Base')
).properties(
    width=150, 
    title='Total de aplicações da Segunda Dose'
)

alt.Chart(...)

In [38]:
doseUnicaTotal = pd.DataFrame({
    'base': ['Municipal', 'Federal'],
    'total': [dm_vacinadosTotalPorDose['aplicacoes'][2], df_vacinadosTotalPorDose['aplicacoes'][2]]
})

alt.Chart(doseUnicaTotal).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Vacinados'),
    alt.Color('base:N', title='Base')
).properties(
    width=150,
    title='Total de aplicações da Dose Única'
)

alt.Chart(...)

## Comparando o total aplicações da vacinas contra o covid-19.

In [39]:
df_vacinadosTotal = dados_federais_campina['paciente_id'].count()

In [40]:
dm_vacinadosTotal = dados_municipais_campina['vacinadosTotal'][-1] - dados_municipais_campina['vacinadosTotal'][0]

In [41]:
vacinadosTotal = pd.DataFrame({
    'base': ['Municipal', 'Federal'],
    'total': [dm_vacinadosTotal, df_vacinadosTotal]
})

alt.Chart(vacinadosTotal).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Vacinados'),
    alt.Color('base:N', title='Base')
).properties(
    width=150,
    title='Total de Vacinas Aplicadas'
)

alt.Chart(...)

# Testando os dados

## Quantidade de doses aplicadas por Vacina

In [42]:
vacinasAplicadas = dados_federais_campina['vacina_codigo'].value_counts()
vacinasAplicadas = pd.DataFrame({
    'vacina': vacinasAplicadas.index.tolist(),
    'quantidade': vacinasAplicadas.values.tolist()
})

In [43]:
alt.Chart(vacinasAplicadas).mark_bar(size=30).encode(
    alt.X('vacina:N', title='Vacina Aplicada'),
    alt.Y('quantidade:Q', title='Quantidade de Aplicações'),
    alt.Color('vacina:N', title='Vacinas')
).properties(
    width=600,
    title='Total de Dose Aplicadas por Vacina'
)

alt.Chart(...)

In [44]:
print(
    len(df_1DosePorDia),
    len(dm_1DosePorDia)
)

91 91
